In [2]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
from torch.nn.utils.rnn import pad_sequence
from sklearn.metrics import confusion_matrix,precision_score,recall_score
from torch.nn import functional as F
import numpy as np
import pandas as pd


In [3]:
b= np.load("/mnt/sd1/jhansi/interns/chaithra/MS/sal_project/Features/test/new_test_audio_feats.npz", allow_pickle=True)['audio_features']


In [4]:
c=0
for i,audio in enumerate(b):
    if len(np.array(audio).shape)==3:
        continue
    else:
        print(i)
    

24
25


In [5]:

# Load the labels
test_labels = np.load("/mnt/sd1/jhansi/interns/chaithra/MS/sal_project/Features/test/test_c_labels.npz", allow_pickle=True)['labels']

# Remove labels at indices 24 and 25
indices_to_remove = [24, 25]
test_labels = np.delete(test_labels, indices_to_remove, axis=0)

# Print the updated length


In [6]:
a=np.load("/mnt/sd1/jhansi/interns/chaithra/MS/sal_project/Features/train/train_audio_feats.npz", allow_pickle=True)['audio_features']
b= np.load("/mnt/sd1/jhansi/interns/chaithra/MS/sal_project/Features/test/test_audio_feats.npz", allow_pickle=True)['audio_features']

t1=[np.squeeze(np.array(audio), axis=1) for audio in a]
t2 = [
    np.squeeze(np.array(audio), axis=1)
    for audio in b
    if len(np.array(audio).shape) == 3
]

train_audio_features=t1
test_audio_features=t2
train_labels = np.load("/mnt/sd1/jhansi/interns/chaithra/MS/sal_project/Features/train/train_c_labels.npz", allow_pickle=True)['labels']


In [7]:
train_audio_features_tensors = [torch.tensor(f) for f in train_audio_features]
# Now pad the sequences to ensure uniform length
padded_train_audio_features = pad_sequence(train_audio_features_tensors, batch_first=True)

# Print the shape of the padded tensor
print(padded_train_audio_features.shape)

test_audio_features_tensors = [torch.tensor(f) for f in test_audio_features]
padded_test_audio_features = pad_sequence(test_audio_features_tensors, batch_first=True)
print(padded_test_audio_features.shape)


torch.Size([104, 69, 256])
torch.Size([30, 63, 256])


In [8]:
train_labels = np.array(train_labels, dtype=np.int64)
test_labels = np.array(test_labels, dtype=np.int64)


In [ ]:
from imblearn.over_sampling import SMOTE
import torch
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
labels_t = np.squeeze(train_labels)  # Converts to 1D if it's in shape (n_samples, 1)

# Flatten the audio features to 2D (n_samples, sequence_length * num_features)
flattened_train_audio_features = padded_train_audio_features.view(padded_train_audio_features.size(0), -1)
print(flattened_train_audio_features.shape)  # Should be [n_samples, 63 * 256]

# Apply SMOTE to balance the data
smote = SMOTE(random_state=42)
balanced_features, balanced_labels = smote.fit_resample(flattened_train_audio_features.numpy(), labels_t)

# Convert the balanced features and labels to PyTorch tensors
balanced_features = torch.tensor(balanced_features).to(device)
balanced_labels = torch.tensor(balanced_labels).to(device)
# Check the current total size and calculate time steps
num_samples = balanced_features.size(0)
num_features = 256  # Assuming this is fixed
time_steps = balanced_features.size(1) // num_features

# Reshape to 3D
balanced_features = balanced_features.view(num_samples, time_steps, num_features)
print(balanced_features.shape)  # Verify the shape



torch.Size([104, 17664])
torch.Size([148, 69, 256])


In [69]:
import torch
from torch.utils.data import DataLoader

# Assuming train_audio_features and test_audio_features have been padded earlier
class AudioDataset(torch.utils.data.Dataset):
    def __init__(self, audio_features, labels):
        self.audio_features = audio_features
        self.labels = labels

    def __len__(self):
        return len(self.audio_features)

    def __getitem__(self, idx):
        # No need to wrap the features and labels again with torch.tensor
        return self.audio_features[idx], torch.tensor(self.labels[idx], dtype=torch.long)

# Create train and test datasets
train_dataset = AudioDataset(padded_train_audio_features, train_labels)
test_dataset = AudioDataset(padded_test_audio_features, test_labels)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=12, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=12, shuffle=False)



In [13]:
import torch
from torch.utils.data import DataLoader

# Assuming train_audio_features and test_audio_features have been padded earlier
class AudioDataset(torch.utils.data.Dataset):
    def __init__(self, audio_features, labels):
        self.audio_features = audio_features
        self.labels = labels

    def __len__(self):
        return len(self.audio_features)

    def __getitem__(self, idx):
        # No need to wrap the features and labels again with torch.tensor
        return self.audio_features[idx], torch.tensor(self.labels[idx], dtype=torch.long)

# Create train and test datasets
train_dataset = AudioDataset(balanced_features, balanced_labels)
test_dataset = AudioDataset(padded_test_audio_features, test_labels)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=12, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=12, shuffle=False)



In [14]:
class AudioBiLSTM(nn.Module):
    def __init__(self, config):
        super(AudioBiLSTM, self).__init__()
        self.num_classes = config['num_classes']
        self.learning_rate = config['learning_rate']
        self.dropout = config['dropout']
        self.hidden_dims = config['hidden_dims']
        self.rnn_layers = config['rnn_layers']
        self.embedding_size = config['embedding_size']
        self.bidirectional = config['bidirectional']

        self.build_model()
        # self.init_weight()

    def init_weight(net):
        for name, param in net.named_parameters():
            if not 'ln' in name:
                if 'bias' in name:
                    nn.init.constant_(param, 0.0)
                elif 'weight' in name:
                    nn.init.xavier_uniform_(param)

    def build_model(self):
        # attention layer
        self.attention_layer = nn.Sequential(
            nn.Linear(self.hidden_dims, self.hidden_dims),
            nn.ReLU(inplace=True))
        self.lstm_net_audio = nn.GRU(self.embedding_size, self.hidden_dims,
                                num_layers=self.rnn_layers, dropout=self.dropout, batch_first=True)

        self.ln = nn.LayerNorm(self.embedding_size)

        # FC
        self.fc_audio = nn.Sequential(
            nn.Dropout(self.dropout),
            nn.Linear(self.hidden_dims, self.hidden_dims),
            nn.ReLU(),
            nn.Dropout(self.dropout),
            nn.Linear(self.hidden_dims, self.num_classes),
          
        )

    def attention_net_with_w(self, lstm_out, lstm_hidden):
        '''
        :param lstm_out:    [batch_size, len_seq, n_hidden * 2]
        :param lstm_hidden: [batch_size, num_layers * num_directions, n_hidden]
        :return: [batch_size, n_hidden]
        '''
        lstm_tmp_out = torch.chunk(lstm_out, 2, -1)
        # h [batch_size, time_step, hidden_dims]
        h = lstm_tmp_out[0] 
        #         h = lstm_out
        # [batch_size, num_layers * num_directions, n_hidden]
        lstm_hidden = torch.sum(lstm_hidden, dim=1)
        # [batch_size, 1, n_hidden]
        lstm_hidden = lstm_hidden.unsqueeze(1)
        # atten_w [batch_size, 1, hidden_dims]
        atten_w = self.attention_layer(lstm_hidden)
        # m [batch_size, time_step, hidden_dims]
        m = nn.Tanh()(h)
        # atten_context [batch_size, 1, time_step]
       # print(atten_w.shape, m.transpose(1, 2).shape)
        atten_context = torch.bmm(atten_w, m.transpose(1, 2))
        # softmax_w [batch_size, 1, time_step]
        softmax_w = F.softmax(atten_context, dim=-1)
        # context [batch_size, 1, hidden_dims]
        context = torch.bmm(softmax_w, h)
        result = context.squeeze(1)
        return result

    def forward(self, x):
        x = self.ln(x)
        x, _ = self.lstm_net_audio(x)
        x = x.mean(dim=1)
        out = self.fc_audio(x)
        return out


In [23]:
config = {
    'num_classes': 2,
    'dropout': 0.4,
    #0.5
    'rnn_layers': 4,
    #2
    'embedding_size': 256,
    'batch_size': 8,
    'epochs': 10,
    'learning_rate': 6e-4,
    'hidden_dims': 256,
    'bidirectional': False,
    'cuda': False
}


In [24]:
model = AudioBiLSTM(config)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)  # Move model to GPU

# Define optimizer and loss function
optimizer = optim.AdamW(model.parameters(), lr=config['learning_rate'])
criterion = nn.CrossEntropyLoss()

In [25]:
best_accuracy = 0.0


In [26]:
# Training function

def train(epoch, train_loader,val_loader,best_accuracy):
    model.train()
    total_loss = 0
    correct = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        # Get sequence lengths (for packing)
        data = data.to(device)
        target = target.to(device)
        target = target.squeeze() 
        optimizer.zero_grad()
        output = model(data)
        
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
    
    avg_train_loss = total_loss / len(train_loader)
    accuracy = correct / len(train_loader.dataset)
    
    
    model.eval()
    val_loss = 0
    val_correct = 0
    with torch.no_grad():
        for data, target in val_loader:
            data = data.to(device)
            target = target.to(device)
            target = target.squeeze()
            
            output = model(data)
            loss = criterion(output, target)
            val_loss += loss.item()
            
            pred = output.argmax(dim=1, keepdim=True)
            val_correct += pred.eq(target.view_as(pred)).sum().item()

    avg_val_loss = val_loss / len(val_loader)
    val_accuracy = val_correct / len(val_loader.dataset)

    print(f"Train Epoch: {epoch}, Loss: {avg_train_loss:.4f}, Accuracy: {accuracy:.4f}")
    print(f"Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

    if val_accuracy > best_accuracy:
        best_accuracy = val_accuracy
        torch.save(model.state_dict(), '/mnt/sd1/jhansi/interns/chaithra/MS/sal_project/Classification/Models/audio/audio_best_model_5.pth')  # Save the best model
    return best_accuracy

In [27]:
for epoch in range(1, config['epochs'] + 1):
    best_accuracy=train(epoch, train_loader,test_loader,best_accuracy)
    print(f"Best model saved with accuracy: {best_accuracy:.4f}")


/tmp/ipykernel_69106/4174753997.py:15: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return self.audio_features[idx], torch.tensor(self.labels[idx], dtype=torch.long)


Train Epoch: 1, Loss: 0.6915, Accuracy: 0.4595
Validation Loss: 0.7027, Validation Accuracy: 0.3667
Best model saved with accuracy: 0.3667
Train Epoch: 2, Loss: 0.5006, Accuracy: 0.8514
Validation Loss: 0.8816, Validation Accuracy: 0.5667
Best model saved with accuracy: 0.5667
Train Epoch: 3, Loss: 0.2201, Accuracy: 0.9595
Validation Loss: 2.1828, Validation Accuracy: 0.6000
Best model saved with accuracy: 0.6000
Train Epoch: 4, Loss: 0.0686, Accuracy: 0.9730
Validation Loss: 2.2497, Validation Accuracy: 0.4000
Best model saved with accuracy: 0.6000
Train Epoch: 5, Loss: 0.0895, Accuracy: 0.9730
Validation Loss: 1.5651, Validation Accuracy: 0.5333
Best model saved with accuracy: 0.6000
Train Epoch: 6, Loss: 0.0315, Accuracy: 0.9865
Validation Loss: 1.6471, Validation Accuracy: 0.6667
Best model saved with accuracy: 0.6667
Train Epoch: 7, Loss: 0.0078, Accuracy: 1.0000
Validation Loss: 1.9473, Validation Accuracy: 0.6000
Best model saved with accuracy: 0.6667
Train Epoch: 8, Loss: 0.001

In [33]:
from sklearn.metrics import f1_score, confusion_matrix

# Evaluation function
def evaluate(test_loader):
    model.eval()
    correct = 0
    total_loss = 0.0  
    total=0
    all_preds = []  # To store all predictions
    all_targets = []  # To store all true labels
    with torch.no_grad():
        for data, target in test_loader:
            data = data.to(device)
            target = target.to(device)
            target = target.squeeze()
            
            output = model(data)
            
            total += target.size(0)
            loss = criterion(output, target)
            total_loss += loss.item()
            
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            
            all_preds.extend(pred.cpu().numpy().flatten())
            all_targets.extend(target.cpu().numpy().flatten())
            
        accuracy = correct / total
        average_loss = total_loss / len(test_loader)
        results_df = pd.DataFrame({
        'Predicted': all_preds,
        'Target': all_targets
        })
    
        #print("\nPredicted vs Target:")
        #print(results_df)
        
        f1 = f1_score(all_targets, all_preds, average='weighted')
        cm = confusion_matrix(all_targets, all_preds)
        precision = precision_score(all_targets, all_preds, average='weighted')
        recall = recall_score(all_targets, all_preds, average='weighted')
        
    print(f"Loss: {average_loss:.4f}, Accuracy: {accuracy:.4f}")
    print(f"F1 Score: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}")
    
    print("Confusion Matrix:")
    print(cm)
    return results_df

In [ ]:
############# DATA Aug  #################

model.load_state_dict(torch.load('/mnt/sd1/jhansi/interns/chaithra/MS/sal_project/Classification/Models/audio/audio_best_model_5.pth', weights_only=True))
model.eval()  # Set to evaluation mode

# Evaluate on the test set
accuracy = evaluate(test_loader)


Predicted vs Target:
    Predicted  Target
0           0       0
1           0       0
2           0       0
3           0       1
4           0       1
5           1       1
6           0       1
7           1       1
8           0       0
9           0       1
10          1       1
11          0       0
12          0       0
13          1       0
14          0       0
15          0       0
16          1       1
17          1       0
18          0       1
19          0       0
20          0       1
21          1       0
22          0       0
23          0       1
24          0       0
25          0       0
26          0       0
27          0       0
28          1       1
29          0       0
Loss: 1.6471, Accuracy: 0.6667
F1 Score: 0.6500, Precision: 0.6591, Recall: 0.6667
Confusion Matrix:
[[15  3]
 [ 7  5]]
